In [1]:
import pandas as pd
import os
import lightgbm as lgb
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import time

from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor

import keras
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


In [2]:
price = pd.read_csv('./transpose-123 - transpose.csv')
supply = pd.read_excel('../data_2020.xlsx')
supply['년도'] = supply['거래년월일'].apply(lambda x: str(x)[:4]).astype(str)
supply['월'] = supply['거래년월일'].apply(lambda x: str(x)[4:6]).astype(str)
supply['일'] = supply['거래년월일'].apply(lambda x: str(x)[6:8]).astype(str)
supply['거래량'] = supply['거래량'].apply(lambda x: ''.join(str(x).split(','))).astype(int)
sup_total = supply.groupby(['년도','월','일'], as_index=False).sum()
sup_total = sup_total[:-2]
sup_total.head()

,년도,월,일,시장코드,거래량,품목코드
0,2020,01,02,371501.0,6,120701.0
1,2020,01,03,690014.0,1286,362103.0
2,2020,01,04,1071509.0,134,482804.0
3,2020,01,06,371501.0,9,120701.0
4,2020,01,07,230003.0,12,120701.0


In [3]:
price['년도'] = price['년도'].apply(lambda x: str(x))
price['월'] = price['월/일'].apply(lambda x: str(x)[0:2]).astype(str)
price['일'] = price['월/일'].apply(lambda x: str(x)[3:5]).astype(str)
price.head()

,년도,월/일,평균,월,일
0,2000,01/03,"103,000",01,03
1,2000,01/04,"97,000",01,04
2,2000,01/05,"95,000",01,05
3,2000,01/06,"98,000",01,06
4,2000,01/07,"98,000",01,07


In [4]:
df = pd.merge(price, sup_total, on=['년도','월','일'])
df.head()

,년도,월/일,평균,월,일,시장코드,거래량,품목코드
0,2020,01/02,"63,400",01,02,371501.0,6,120701.0
1,2020,01/03,"60,000",01,03,690014.0,1286,362103.0
2,2020,01/06,"58,800",01,06,371501.0,9,120701.0
3,2020,01/07,"56,200",01,07,230003.0,12,120701.0
4,2020,01/09,"51,200",01,09,601502.0,18,241402.0


In [5]:
df2 = df[['년도','월','일','평균','거래량']]
df2['년도'] = df2['년도'].apply(lambda x: int(x))
df2['월'] = df2['월'].apply(lambda x: int(x))
df2['일'] = df2['일'].apply(lambda x: int(x))
df2['평균'] = df2['평균'].apply(lambda x: ''.join(str(x).split(','))).astype(int) 

df2

/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,년도,월,일,평균,거래량
0,2020,1,2,63400,6
1,2020,1,3,60000,1286
2,2020,1,6,58800,9
3,2020,1,7,56200,12
4,2020,1,9,51200,18
5,2020,1,10,51200,496
6,2020,1,13,56200,912
7,2020,1,14,60800,457
8,2020,1,15,63600,936
9,2020,1,16,61800,705


In [6]:
a = list(df2['평균'])
df2['d+1'] = a
a.insert(0,0)
a = a[:-1]

df2['d-0'] = a
a.insert(0,0)
a = a[:-1]

df2['d-1'] = a
a.insert(0,0)
a = a[:-1]

df2['d-2'] = a
a.insert(0,0)
a = a[:-1]

df2['d-3'] = a
a.insert(0,0)
a = a[:-1]

df2['d-4'] = a
a.insert(0,0)
a = a[:-1]

df2['d-5'] = a
a.insert(0,0)
a = a[:-1]

df2['d-6'] = a
a.insert(0,0)
a = a[:-1]

df2['d-7'] = a

del df2['평균']
df2 = df2[8:]
df2 = df2.reset_index(drop=True)
df2.head(10)

y = df2['d+1']
del df2['d+1']

/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
df2.head(10)

,년도,월,일,거래량,d-0,d-1,d-2,d-3,d-4,d-5,d-6,d-7
0,2020,1,15,936,60800,56200,51200,51200,56200,58800,60000,63400
1,2020,1,16,705,63600,60800,56200,51200,51200,56200,58800,60000
2,2020,1,17,62,61800,63600,60800,56200,51200,51200,56200,58800
3,2020,1,20,270,62400,61800,63600,60800,56200,51200,51200,56200
4,2020,1,21,555,68200,62400,61800,63600,60800,56200,51200,51200
5,2020,1,22,386,77400,68200,62400,61800,63600,60800,56200,51200
6,2020,1,23,65,78800,77400,68200,62400,61800,63600,60800,56200
7,2020,1,30,243,77200,78800,77400,68200,62400,61800,63600,60800
8,2020,1,31,253,69400,77200,78800,77400,68200,62400,61800,63600
9,2020,2,3,150,65600,69400,77200,78800,77400,68200,62400,61800


In [9]:
seed_pred_result = []
rmse_result = []
for i in range(10):
    rand_num = i
    x_train, x_test,y_train, y_test = train_test_split(df2, y, test_size=0.9, random_state=rand_num)
    print(y_test.shape)
    models = [{'name':'XGBoost'},
              {'name': 'GradientBoosting'},
              {'name':'LightGBM'},
              {'name': 'Lasso'}]

    result = []
    rmse = []
    result.append(y_test)

    for m in models:
        loaded_model = pickle.load(open("pima_{}_{}.pickle.dat".format(i, m['name']), "rb"))
        predictions = loaded_model.predict(x_test.values) 
        result.append(predictions)
        rmse.append({m['name']:sqrt(mean_squared_error(y_test, predictions))})
    dict = {'y': result[0], 'xgboost': result[1], 'gboost': result[2], 'lightgbm':result[3], 'lasso':result[4]}

    df_res = pd.DataFrame(dict)
    df_res['average'] = df_res[['xgboost', 'gboost', 'lightgbm', 'lasso']].apply(lambda x: round(x.mean()), axis=1)
    rmse.append({'average':sqrt(mean_squared_error(y_test, df_res['average']))})

    rmse_result.append(rmse)
    seed_pred_result.append(df_res)    

(79,)
(79,)
(79,)
(79,)
(79,)
(79,)
(79,)
(79,)
(79,)
(79,)


In [16]:
rmse_result[:4]

[[{'XGBoost': 9511.555646302673},
  {'GradientBoosting': 8028.68600414758},
  {'LightGBM': 7809.594149408684},
  {'Lasso': 7469.757733424508},
  {'average': 7902.912551551381}],
 [{'XGBoost': 7916.502180207416},
  {'GradientBoosting': 8400.394991898844},
  {'LightGBM': 8473.23216815017},
  {'Lasso': 7477.436165521469},
  {'average': 7744.933863950165}],
 [{'XGBoost': 7733.372885616587},
  {'GradientBoosting': 7836.3228066568145},
  {'LightGBM': 7799.172654523125},
  {'Lasso': 7492.928111264407},
  {'average': 7532.101077758424}],
 [{'XGBoost': 8374.21379311465},
  {'GradientBoosting': 8468.60778881898},
  {'LightGBM': 8097.034669643283},
  {'Lasso': 7587.441330938508},
  {'average': 7855.480861769078}]]

In [11]:
seed_pred_result[0].head(5)

,y,xgboost,gboost,lightgbm,lasso,average
2,62400,64180.847656,63137.935801,63593.310790,62309.556450,63305.0
13,49600,49901.468750,49642.030392,48054.583210,46658.866077,48564.0
53,147600,101980.765625,98880.109473,105404.689570,98396.547944,101166.0
41,121400,116216.234375,117404.480500,112172.538739,120854.572027,116662.0
66,64600,64459.765625,61071.353341,62186.583334,63038.811356,62689.0


In [12]:
seed_pred_result[1].head(5)

,y,xgboost,gboost,lightgbm,lasso,average
38,131200,134376.937500,135699.676640,138084.821089,132708.425900,135217.0
44,123600,123404.257812,124417.173037,121943.524249,122630.487912,123099.0
27,73750,72420.039062,74648.942340,74113.518209,74699.069112,73970.0
40,121000,124330.000000,120375.041973,115726.180474,118856.450404,119822.0
36,128000,119758.218750,121952.051830,119641.263699,122847.001373,121050.0


In [13]:
seed_pred_result[2].head(5)

,y,xgboost,gboost,lightgbm,lasso,average
44,123600,125535.031250,122010.180248,121956.469141,122699.358121,123050.0
35,120000,108393.296875,112773.065477,114633.806372,113891.036280,112423.0
82,42000,38810.582031,38852.540536,39498.682661,39760.378310,39231.0
0,63600,60742.519531,61240.438657,61166.616295,62288.899811,61360.0
11,47800,55125.125000,53461.359508,52447.945495,51955.392530,53247.0


In [14]:
seed_pred_result[3].head(5)

,y,xgboost,gboost,lightgbm,lasso,average
62,76200,79388.796875,79989.719171,81970.235346,77933.016838,79820.0
75,42000,42503.730469,41413.066174,41993.582109,40971.704746,41721.0
9,55400,66962.335938,67441.585243,67569.461261,65145.888969,66780.0
45,123600,129319.210938,122434.842185,123778.512003,122119.877770,124413.0
34,112000,110317.507812,109530.333512,108221.829079,110281.379272,109588.0


In [15]:
seed_pred_result[4].head(5)

,y,xgboost,gboost,lightgbm,lasso,average
76,40000,42707.355469,42149.713639,42874.628810,42656.956321,42597.0
83,45600,43166.125000,42270.454202,43234.769793,43423.060893,43024.0
14,55200,49029.089844,49974.085532,49517.637752,49969.341666,49623.0
35,120000,110070.437500,110627.798408,109129.347533,114090.725661,110980.0
27,73750,74067.046875,74829.793384,74543.243360,74592.144227,74508.0
